# Modern Algebra RSA Project - Riddles & Solutions

Answers with explanations to the project's riddles.

In [44]:
def immediately_invoke(func):
    """
    Immediately invoke the provided function to prevent a cell's local variables from polluting 
    """
    import sys
    from importlib import reload
    if 'number_theory_functions' in sys.modules:
        reload(sys.modules['number_theory_functions'])
    if 'rsa_functions' in sys.modules:
        reload(sys.modules['rsa_functions'])
    return func()

## 1. Loki & Spiderman

### Riddle

Loki wants to be paid 1,000,000\$ exactly. Spiderman only has coins that are worth 797\$ each. Loki suggests providing change but only with 5279\$ bills.

Is there a chance for such a transaction? How?

### Answer

The question is if there is a solution to $797 * x = 1000000 (mod 5279)$. We need to find the modular inverse.

In [38]:
from number_theory_functions import modular_inverse

@immediately_invoke
def loki_and_spiderman():
    loki_bill = 5279
    goal = 1_000_000
    spiderman_coin = 797

    goal_inverse = modular_inverse(goal, loki_bill)
    x = goal_inverse and modular_inverse((goal_inverse * spiderman_coin) % loki_bill, loki_bill)
    if x is None:
        print("Not Possible")
    else:
        change = x * spiderman_coin - goal
        assert change >= 0
        assert change % loki_bill == 0
        change_bills = change // loki_bill
        print(f"Spiderman can pay with {x} coins and get {change_bills} bills ({change}$) as change")
    return x

Spiderman can pay with 3977 coins and get 411 bills (2169669$) as change


## 2. Hundreds Digit

### Riddle

What is the hundredth digit of the number $(123456)^{(7896543^{(74365753)})}$

### Answer

We want to find the result of integer division of $123456^{(7896543^{74365753})} (mod 1000)$ by $100$.

According to the conclusions from Lagrange's theorem, if $G$ is a finite group and $a \in G$, then $o(a) | |G|$.

$G = Z_{1000}$ so,

$$(123456 (mod 1000))^{(7896543^{74365753} (mod 1000))} (mod 1000)$$

In [39]:
from number_theory_functions import modular_exponent

@immediately_invoke
def hundreds_digit():
    exp = modular_exponent(7896543, 74365753, 1000)
    hundreds_digit = modular_exponent(123456, exp, 1000) // 100
    print(f"The hundreds digit of 123456**(7896543**74365753) is {hundreds_digit}")
    return hundreds_digit

The hundreds digit of 123456**(7896543**74365753) is 0


## 3. cipher 42

### Riddle

Given a public key $e = 3499$, $N = 12215009$, and an encrypted message $42$. Crack the code.
(Hint: You should first guess the private key, given a magic Crystal ball or wolfram alpha)

### Answer

We know those params:

$$e = 3499$$

$$N = 12215009$$

$$Me = M^e(mod(\varphi(N))) = 42$$

First, we calculate $\varphi(N)$ with an online Euler's Function calculator:
$\varphi(N) = 12208020$

Now, we use wolfram alpha to calculate d:
$d = 5425399$

Now, we calculate the cipher by raising $42$ to the power of $d$, mod $\varphi(N)$:
`M = modular_exponent(42, 5425399, 12208020)`

In [40]:
from number_theory_functions import modular_exponent

@immediately_invoke
def cipher_42():
    e = 3499
    phi_N = 12208020
    Me = 42

    M = modular_exponent(Me, e, phi_N)
    print(f"Cracked the code! The ciphered code is {M}")

    return M

Cracked the code! The ciphered code is 6014508


## 4. Finding the Inverse Function

### Riddle

Look at the function $E(x) = x^e (mod N)$ where $e = 11$ and $N = 991$.
Is $E$ inversible? If so, calculate $D(y) = E^{-1}(x)$.

### Answer

Based on _RSA_'s algorithm, we're looking for $D(y) = y^d (mod\space N)$ such that $ed \equiv 1 (mod\space \varphi(N))$.

$N = 991$ is a prime number $\Rightarrow \varphi(N) = N - 1$.

We'll try to find such $d$:

In [43]:
from number_theory_functions import modular_inverse

@immediately_invoke
def inverse_exponent():
    e = 11
    N = 991

    d = modular_inverse(e, N - 1)

    if d is None:
        print("D(y) does not exist")
    else:
        print("D(y) := (y ** {d}) % {N}")

    return d

D(y) does not exist


## 5. Encrypt

### Riddle

Given $q = 6841$ and $p = 7919$, choose a message and a public key $e$ and encrypt the message.
(Show your original message, the public key and the encrypted message)

### Answer

We can use our RSA implementation to find a valid public key $e$ and encrypt the message.

In [59]:
@immediately_invoke
def encrypt():
    from rsa_functions import RSA

    p = 7919
    q = 6841

    rsa = RSA.from_primes(p, q)

    plain = 42
    encrypted = rsa.encrypt(plain)
    assert encrypted is not None
    assert plain == rsa.decrypt(encrypted)

    print(f"e = {rsa.public_key[1]}")
    print(f"the plain message: {plain}")
    print(f"the encrypted message: {rsa.encrypt(plain)}")

    return encrypted

e = 33551899
the plain message: 42
the encrypted message: 6083595
